In [1]:
import os


In [2]:
os.chdir("../")

In [3]:
pwd

'/home/azureuser/chicken_disease_classification'

In [4]:
import tensorflow as tf

2023-06-15 17:44:01.988632: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 17:44:02.248536: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 17:44:02.251391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 17:44:05.670341: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [10]:
from chicken_disease.constants import *
from chicken_disease.utils.common import read_yaml , create_directories ,save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [14]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-06-15 18:02:14,785: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-06-15 18:02:14,792: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-15 18:02:14,794: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


2023-06-15 18:02:15.385945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 18:02:15.775958: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 308281344 exceeds 10% of free system memory.
2023-06-15 18:02:16.229485: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 308281344 exceeds 10% of free system memory.
2023-06-15 18:02:18.341949: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154140672 exceeds 10% of free system memory.
2023-06-15 18:02:19.178494: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154140672 exceeds 10% of free system memory.


1/5 [=====>........................] - ETA: 58s - loss: 9.9341e-09 - accuracy: 1.0000

2023-06-15 18:02:29.975734: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 308281344 exceeds 10% of free system memory.


5/5 [==============================] - 68s 13s/step - loss: 6.1655 - accuracy: 0.5000
[2023-06-15 18:03:37,615: INFO: common: json file saved at: scores.json]
